In [ ]:
import gradio
import pandas
import collections

In [ ]:
# Constraints: Only supports english and turkish character set
letters = 'ABCÇDEFGĞHIİJKLMNOÖPQRSŞTUÜVWXYZ'
uppercase_letters = 'ABCÇDEFGĞHIİJKLMNOÖPQRSŞTUÜVWXYZ'
lowercase_letters = 'abcçdefgğhii̇jklmnoöpqrsştuüvwxyz'

# For unit testing
test_suite = unittest.TestSuite()

In [ ]:
def removeNonAlphabeticCharacters(text):

    modified_text = ''
    for character in text:
        if (uppercase_letters.find(character) != -1 or lowercase_letters.find(character) != -1):
            modified_text += character

    return modified_text

# Test code
# Output: önder
# text = 'ö n,d1!e;r'
# print(removeNonAlphabeticCharacters(text))

In [ ]:
# Input: Önder Görmez
#
# Output:
# pandas.DataFrame({
#     'letter': ['d', 'e', 'g', 'm', 'n', 'ö', 'r', 'z',],
#     'count':  [  1,   2,   1,  1,    1,   2,   2,   1,]
# })
def calculateLetterCount(text):

    # It is assumed that non-alphabetic characters are not encrypted.
    text = removeNonAlphabeticCharacters(text)

    letter_counter_dict = collections.Counter(text)
    letter_counter_df = pandas.DataFrame.from_dict(letter_counter_dict, orient='index').reset_index()
    letter_counter_df.columns = ["letter", "count"]
    letter_counter_df = letter_counter_df.sort_values(by=['count'], ascending=False)
    print(letter_counter_df)

    return letter_counter_df

In [ ]:
# Constraints: Only supports english and turkish character set
def encryptText(plaintext, letter_shift_count):

    
    start_character_idx = letters.index('A')
    end_character_idx = letters.index('Z')
    
    print('Plaintext:')
    print(plaintext)

    encrypted = ''
    
    for character in plaintext:

        # Exception case for other non-letter characters (exp: ', ;:.!')
        if (letters.find(character) == -1):
            encrypted += character
            continue

        new_character_index = int(letters.index(character) + letter_shift_count)

        if (new_character_index > end_character_idx):
            new_character_index = start_character_idx + (new_character_index - end_character_idx - 1)
        
        encrypted += letters[new_character_index]

    print('Encrypted:')
    print(encrypted)

    return encrypted

# Test code
# plaintext = "Hello, world!"
# letter_shift_count = 1
# encryptText(plaintext, letter_shift_count)

In [ ]:
def calculateCryptoAnalysisPerformance(plaintext, deciphered_text):
    match_count = 0

    for idx in range(len(plaintext)):
        if (plaintext[idx] == deciphered_text[idx]):
            match_count += 1

    accuracy_percentage = (match_count / len(plaintext)) * 100

    accuracy_str = str(round(accuracy_percentage, 2)) + '% Accuracy'
    print(accuracy_str)

    return accuracy_str

# Test code
# Output: 33.33% Accuracy

# plaintext = 'Hel'
# deciphered_text = 'Hdo'
# calculateCryptoAnalysisPerformance(plaintext, deciphered_text)

In [ ]:
def decipherText(ciphertext, ciphertext_letter_count, reference_text_letter_count):

    ciphertext_array = ''.join(map(str, ciphertext_letter_count['letter'].values))
    reference_text_array = ''.join(map(str, reference_text_letter_count['letter'].values))

    deciphered_text = ''
    for encrypted_character in ciphertext:

        # Skip non-letter characters (exp: ', ;:.!')
        if (uppercase_letters.find(encrypted_character) == -1 and lowercase_letters.find(encrypted_character) == -1):
            deciphered_text += encrypted_character
            continue

        index = ciphertext_array.index(encrypted_character)
        # TODO: Referans text den büyükse burada kontrol et.
        deciphered_text += reference_text_array[index]

    print("ciphertext: " + ciphertext_array)
    print("deciphered_text: " + deciphered_text)

    return deciphered_text

# Test code
# ciphertext = 'poefs ğpsnfa'
# 
# ciphertext_letter_count = pandas.DataFrame({
#     'letter': ['e', 'f', 'ğ', 'n', 'o', 'p', 's', 'a', ' '],
#     'count':  [  1,   2,   1,  1,    1,   2,   2,   1,  1 ]
# })
# 
# reference_text_letter_count = pandas.DataFrame({
#     'letter': ['d', 'e', 'g', 'm', 'n', 'ö', 'r', 'z', ' '],
#     'count':  [  1,   2,   1,  1,    1,   2,   2,   1,  1]
# })
# 
# decipherText(ciphertext, ciphertext_letter_count, reference_text_letter_count)

In [ ]:
DELIMITER = ";"
SEPARATOR = "-->"

def postProcessor(deciphered_text, post_processing_texts):

    if len(post_processing_texts) == 0:
        return deciphered_text

    # Remove whitspaces
    post_processing_texts = post_processing_texts.upper().replace(" ","")

    str_array = post_processing_texts.split(DELIMITER)

    for str in str_array:
        from_text, to_text = str.split(SEPARATOR)
        deciphered_text = deciphered_text.replace(from_text, to_text)

    return deciphered_text

# Test code
deciphered_text = 'ÖZNER DÖRGEM'
post_processing_texts = 'n --> d;z --> N'

postProcessor(deciphered_text, post_processing_texts)

In [ ]:
def entryFunction(plaintext, letter_shift_count, reference_text, post_processing_texts):

    plaintext = plaintext.upper()
    reference_text = reference_text.upper()

    # User input validation part
    if (len(plaintext) == 0):
        raise gradio.Error("Plaintext character length must be greater than zero")

    if (letter_shift_count < 1 or letter_shift_count > 15):
        raise gradio.Error("Letter Shift Count must be between 1 and 15")

    if (len(reference_text) == 0):
        raise gradio.Error("Reference Text character length must be greater than zero")

    # Ciphertext: Encrypted text transformed from plaintext using an encryption algorithm.
    ciphertext = encryptText(plaintext, letter_shift_count)

    ciphertext_letter_count = calculateLetterCount(ciphertext)
    reference_text_letter_count = calculateLetterCount(reference_text)

    deciphered_text = decipherText(ciphertext, ciphertext_letter_count, reference_text_letter_count)
    crypto_analysis_performance = calculateCryptoAnalysisPerformance(plaintext, deciphered_text)

    post_processed_text = postProcessor(deciphered_text, post_processing_texts)
    post_processed_performance = calculateCryptoAnalysisPerformance(plaintext, post_processed_text)

    return ciphertext, deciphered_text, crypto_analysis_performance, post_processed_text, post_processed_performance, ciphertext_letter_count, reference_text_letter_count;

In [ ]:
# Plaintext: Text that is not computationally tagged, specially formatted, or written in code.
plaintext_textbox = gradio.Textbox(label="Plaintext (Unencrypted Text)", placeholder="Enter the words to be encrypted...")

letter_shift_count_number = gradio.Number(label="Letter Shift Count")

reference_text_textbox = gradio.Textbox(label="Reference Text", placeholder="Enter the reference text to be used for letter frequency estimation...")

post_processing_textbox = gradio.Textbox(label="Post Processing (Fine Tuning)", placeholder="Replace deciphered text letters or words (Exp. Inputs: n-->a; hgd-->and; od-->or).")

# Ciphertext: Encrypted text transformed from plaintext using an encryption algorithm.
ciphertext_textbox = gradio.Textbox(label="Ciphertext (Encrypted Text)", placeholder="The result will be here after submission...")

deciphered_text_textbox = gradio.Textbox(label="Deciphered Text", placeholder="The result will be here after submission...")

crypto_analysis_performance_textbox = gradio.Textbox(label="Crypto Analysis Performance", placeholder="The result will be here after submission...")

post_processed_text_textbox = gradio.Textbox(label="Post Processed Text", placeholder="The result will be here after submission...")

post_processed_performance_textbox = gradio.Textbox(label="Post Processed Performance", placeholder="The result will be here after submission...")

test_data_df = calculateLetterCount('Test Data')

ciphertext_letter_count = gradio.BarPlot(
        test_data_df,
        x="letter",
        y="count",
        title="Ciphertext (Encrypted Text) Letter Count",
        tooltip=['letter', 'count']
    )

reference_text_letter_count = gradio.BarPlot(
        test_data_df,
        x="letter",
        y="count",
        title="Reference Text Letter Count",
        tooltip=['letter', 'count']
    )

demo = gradio.Interface(fn=entryFunction,
                        inputs=[plaintext_textbox, letter_shift_count_number, reference_text_textbox, post_processing_textbox],
                        outputs=[ciphertext_textbox, deciphered_text_textbox, crypto_analysis_performance_textbox, post_processed_text_textbox,
                                 post_processed_performance_textbox, ciphertext_letter_count, reference_text_letter_count])

# TODO: Uncomment demo.launch(share=True)
demo.launch(share=True)
# server_port=8080
demo.launch()